<a href="https://colab.research.google.com/github/changyuhsin1999/Nintendo_Switch_Game_Recommendation/blob/main/recommendation_by_game_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Nintendo_Switch_Game_Recommendation" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/Nintendo_Switch_Game_Recommendation.git'
!git clone "{git_path}"

Cloning into 'Nintendo_Switch_Game_Recommendation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 1.87 MiB | 6.95 MiB/s, done.


In [3]:
import urllib
import zipfile
import time

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
raw_df = pd.read_csv('/content/Nintendo_Switch_Game_Recommendation/switch-games.csv')
clean_df = raw_df.loc[:,['id','title','developer','genre','rating']].dropna()
final_df = clean_df[clean_df.rating != 0.0]
final_df.head()

,id,title,developer,genre,rating
3,e2f10932,Temtem,Crema Games,Action Adventure RPG,7.8
5,4377dbf1,Splatoon 3,Nintendo,Action,9.1
6,751a7bbf,JoJos Bizarre Adventure: All-Star Battle R,CyberConnect2,Fighting,7.0
8,438bc630,Pac-Man World Re-PAC,Bandai Namco,Action Adventure Platformer,6.9
9,7796d27e,Teenage Mutant Ninja Turtles: The Cowabunga Co...,Digital Eclipse,Action Arcade Fighting Platformer,6.9


In [5]:
final_df['content'] = final_df['title'].astype(str) + ' ' + final_df['developer'].astype(str) + ' ' + final_df['genre'] + ' ' + final_df['rating'].astype(str) + ' '
final_df['content'] = final_df['content'].fillna('')
final_df['tokenized_content'] = final_df['content'].apply(simple_preprocess)

<ipython-input-5-e7678d3c7b24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['title'].astype(str) + ' ' + final_df['developer'].astype(str) + ' ' + final_df['genre'] + ' ' + final_df['rating'].astype(str) + ' '
<ipython-input-5-e7678d3c7b24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].fillna('')
<ipython-input-5-e7678d3c7b24>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [6]:
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)
model.build_vocab(final_df['tokenized_content'])
model.train(final_df['tokenized_content'], total_examples=model.corpus_count, epochs=10)

(31032, 42530)

In [20]:
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

# Function to compute average word vectors for all games
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features) for tokenized_sentence in corpus]
    return np.array(features)

# Compute average word vectors for all games
w2v_feature_array = averaged_word_vectorizer(corpus=final_df['tokenized_content'], model=model, num_features=100)

# Get the user input
user_game = input("Enter a game title: ")

# Find the index of the user movie
game_index = final_df.index.values[final_df['title'] == user_game]

# Compute the cosine similarities between the user movie and all other movies
user_game_vector = w2v_feature_array[game_index].reshape(1, -1)
similarity_scores = cosine_similarity(user_game_vector, w2v_feature_array)

# Get the top 5 most similar games
similar_games = list(enumerate(similarity_scores[0]))
sorted_similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)[1:6]

# Print the top 5 similar games
for i, score in sorted_similar_games:
    print("{}: {}".format(i, final_df.loc[i, 'title']))

Enter a game title: Splatoon 3
559: Bayonetta 2
383: World End Syndrome
126: Mario Golf: Super Rush
329: Stranded Sails - Explorers of the Cursed Islands
170: Taiko no Tatsujin: Rhythmic Adventure Pack
